# <center>Simulation (Final Draft)

### <center>Members: Wenbin Zhou, Binghe Zhu, Jiazhi He

# Libraries

In [1]:
options (warn = -1) # 不显示warning

library(SIS) # SIS
library(lars) # LARS-LASSO
library(flare)
library(ncvreg) # SCAD
library(fastclime) # Dantzig selector

Loaded lars 1.2


载入需要的程辑包：lattice

载入需要的程辑包：MASS

载入需要的程辑包：Matrix

载入需要的程辑包：igraph


载入程辑包：'igraph'


The following objects are masked from 'package:stats':

    decompose, spectrum


The following object is masked from 'package:base':

    union


Registered S3 methods overwritten by 'fastclime':
  method    from 
  print.sim flare
  plot.sim  flare


载入程辑包：'fastclime'


The following objects are masked from 'package:flare':

    plot.sim, print.sim




# Measure of error

In [2]:
beta_error = function(beta_hat,b,supp_beta_hat){
    
    # beta_hat 为估计出来的参数向量，不含零项
    # b 为真实的参数向量,包含零项
    # supp_beta_hat 为beta_hat的支集指标
    
    ans = sum( (beta_hat - b[supp_beta_hat])^2 ) + sum( (b[-supp_beta_hat])^2 )
    ans = sqrt(ans)
    return(ans)
}

# <center>Realization

# Preparations

In [3]:
set.seed(123) # 保证生成不同的数据集且具有可重复性
    
n = 200 # 样本大小
p = 1000 # 参数维数
x = matrix(rnorm(n*p, mean=0, sd=1), n, p) # 200*1000 独立同分布标准正态自变量
x = scale(x) # 标准化
eps = rnorm(n, mean=0, sd=1.5) # 标准差为1.5的正态噪声

set.seed(456) # 保证生成不同的数据集且具有可重复性
s = 8 # 真实模型大小
    
# 生成beta
u = rbinom(s, 1, 0.4) # 生成 s = 8 个伯努利随机数
z = rnorm(s, mean=0, sd=1) # 生成 s = 8 个正态分布随机数
a = 4*log(n)/sqrt(n) # a的定义
b = ((-1)**(u))*(a + abs(z)) # 长度是 s = 8 的参数向量
    
# 使用beta生成响应变量y
y = x[, 1:s] %*% b + eps  # 回归模型
y = y - mean(y) # 中心化

# my_SIS

In [6]:
my_SIS = function(x,y,nsis){
    x = scale(x)
    omega = t(x) %*% y
    index =  order(abs(omega),decreasing=TRUE)[1:nsis]
    index = sort(index)
    return(index)
}

my_SIS(x,y,nsis=(n/log(n)))

real_SIS = SIS(x, y, family='gaussian', iter = FALSE, nsis=floor(n/log(n)))
real_SIS$sis.ix0

[1]   1   2   3   4   5   6   7   8  57 125 162 167 190 221 228 400 419 427 501
[20] 510 524 635 672 684 686 691 698 741 775 808 819 828 887 893 932 956 986

[1]   1   2   3   4   5   6   7   8  57 125 162 167 190 221 228 400 419 427 501
[20] 510 524 635 672 684 686 691 698 741 775 808 819 828 887 893 932 956 986

# my_ISIS

In [4]:
my_ISIS = function(new,y,nsis){
    x = new
    colnames(x)=1:ncol(x)
    record_index=c()
    for(i in 1:nsis){
        SIS_model = SIS(x, y, family='gaussian', penalty="SCAD",tune="bic",iter = FALSE, nsis=1)
        del_index = SIS_model$sis.ix0
        fit = lm(y~x[,del_index])
        res = resid(fit)
        y = res 
        record_index = c(record_index,as.numeric(colnames(x)[del_index]))
        x = x[,-del_index]
    }
    record_index = sort(record_index)
    return(record_index)
}

my_ISIS(x,y,nsis=(n/log(n)))

real_SIS = SIS(x, y, family='gaussian',penalty="SCAD",tune="bic",iter = TRUE ,nsis=floor(n/log(n)))
real_SIS$ix

[1]   1   2   3   4   5   6   7   8  20  37  38  59  77 125 138 156 184 212 308
[20] 320 345 361 406 412 493 525 573 665 697 701 731 760 790 791 825 861 963

Iter 1 , screening:  1 2 3 4 5 6 7 8 57 125 162 167 228 427 510 684 686 691 741 775 828 887 893 932 
Iter 1 , selection:  1 2 3 4 5 6 7 8 
Iter 1 , conditional-screening:  19 53 77 125 134 212 273 308 313 346 393 412 417 433 443 458 483 495 507 525 577 635 636 673 698 829 883 986 1000 
Iter 2 , screening:  1 2 3 4 5 6 7 8 19 53 77 125 134 212 273 308 313 346 393 412 417 433 443 458 483 495 507 525 577 635 636 673 698 829 883 986 1000 
Iter 2 , selection:  1 2 3 4 5 6 7 8 19 77 125 134 212 273 308 313 346 393 412 417 433 443 458 483 495 507 525 577 635 636 673 698 829 883 986 1000 
Iter 2 , conditional-screening:  716 
Iter 3 , screening:  1 2 3 4 5 6 7 8 19 77 125 134 212 273 308 313 346 393 412 417 433 443 458 483 495 507 525 577 635 636 673 698 716 829 883 986 1000 
Iter 3 , selection:  1 2 3 4 5 6 7 8 19 77 125 134 212 273 308 313 346 393 412 417 433 443 458 483 495 507 525 577 635 636 673 698 716 829 883 986 1000 
Maximum number of variables selected 


[1]    1    2    3    4    5    6    7    8   19   77  125  134  212  273  308
[16]  313  346  393  412  417  433  443  458  483  495  507  525  577  635  636
[31]  673  698  716  829  883  986 1000

# <center> Implements

# n = 200, p = 1000, SIS-SCAD

In [41]:
# 样本量为n = 200，变量数为p = 1000，SIS-SCAD的模拟
errors_SISSCAD_small = c() # 记录每次模拟的误差
model_sizes_SISSCAD_small = c() # 记录每次模拟产生的模型大小
start_time = Sys.time() # 记录SIS-SCAD small开始时间

for(i in 1:50){ # 测试200个数据集
  
    set.seed(123*i) # 保证生成不同的数据集且具有可重复性
    
    n = 200 # 样本大小
    p = 1000 # 参数维数
    x = matrix(rnorm(n*p, mean=0, sd=1), n, p) # 200*1000 独立同分布标准正态自变量
    x = scale(x) # 标准化
    eps = rnorm(n, mean=0, sd=1.5) # 标准差为1.5的正态噪声

    set.seed(456*i) # 保证生成不同的数据集且具有可重复性
    s = 8 # 真实模型大小
    
    # 生成beta
    u = rbinom(s, 1, 0.4) # 生成 s = 8 个伯努利随机数
    z = rnorm(s, mean=0, sd=1) # 生成 s = 8 个正态分布随机数
    a = 4*log(n)/sqrt(n) # a的定义
    b = ((-1)**(u))*(a + abs(z)) # 长度是 s = 8 的参数向量
    
    # 使用beta生成响应变量y
    y = x[, 1:s] %*% b + eps  # 回归模型
    y = y - mean(y) # 中心化
 
    # 建立SIS-SCAD模型
    modelSIS_small = SIS(x,y,family='gaussian',iter = FALSE, nsis=floor(n/log(n)))
    modelSIS_SCAD_small = tune.fit(x[,modelSIS_small$sis.ix0],y,family='gaussian',penalty="SCAD",tune="bic")

    # 计算误差
    b = c(b,rep(0,p-s)) # 真实
    beta_hat = modelSIS_SCAD_small$beta # 估计
    supp_beta_hat = modelSIS_SCAD_small$ix # 支集
    est_error = beta_error(beta_hat,b,supp_beta_hat) # 误差
    errors_SISSCAD_small[i] = est_error # 存储误差
 
    #计算模型大小
    model_sizes_SISSCAD_small[i] = length(supp_beta_hat) # 存储模型大小
    
    # 汇总输出结果
    cat("第",i,"次模拟，误差", errors_SISSCAD_small[i],"，模型大小",model_sizes_SISSCAD_small[i],"\n") # 显示结果
}
end_time = Sys.time() # 记录SIS-SCAD small结束时间

SISSCAD_total_time_small = end_time - start_time # SIS-SCAD small的总耗时
SISSCAD_error_small_median = median(errors_SISSCAD_small) # 误差的中位数
SISSCAD_model_Size_small_median = median(model_sizes_SISSCAD_small) # 模型大小中位数

cat("SIS-SCAD small总耗时",SISSCAD_total_time_small,"秒，误差中位数",SISSCAD_error_small_median,"，模型大小中位数",SISSCAD_model_Size_small_median,"\n")

第 1 次模拟，误差 0.551344 ，模型大小 22 
第 2 次模拟，误差 0.2753522 ，模型大小 10 
第 3 次模拟，误差 0.4372524 ，模型大小 9 
第 4 次模拟，误差 0.3834477 ，模型大小 12 
第 5 次模拟，误差 0.202099 ，模型大小 8 
第 6 次模拟，误差 6.972939 ，模型大小 24 
第 7 次模拟，误差 0.2320908 ，模型大小 8 
第 8 次模拟，误差 0.4949701 ，模型大小 10 
第 9 次模拟，误差 0.4531644 ，模型大小 8 
第 10 次模拟，误差 0.2940667 ，模型大小 10 
第 11 次模拟，误差 0.5959379 ，模型大小 16 
第 12 次模拟，误差 2.425028 ，模型大小 32 
第 13 次模拟，误差 6.81945 ，模型大小 27 
第 14 次模拟，误差 0.3936829 ，模型大小 14 
第 15 次模拟，误差 0.2944618 ，模型大小 8 
第 16 次模拟，误差 0.3136356 ，模型大小 9 
第 17 次模拟，误差 0.3774857 ，模型大小 11 
第 18 次模拟，误差 0.3578761 ，模型大小 11 
第 19 次模拟，误差 0.3749394 ，模型大小 8 
第 20 次模拟，误差 0.224339 ，模型大小 9 
第 21 次模拟，误差 0.3603911 ，模型大小 9 
第 22 次模拟，误差 0.3220718 ，模型大小 8 
第 23 次模拟，误差 2.155401 ，模型大小 27 
第 24 次模拟，误差 0.3724238 ，模型大小 8 
第 25 次模拟，误差 0.2149783 ，模型大小 8 
第 26 次模拟，误差 0.4103303 ，模型大小 11 
第 27 次模拟，误差 0.2880813 ，模型大小 8 
第 28 次模拟，误差 2.089227 ，模型大小 23 
第 29 次模拟，误差 0.2862882 ，模型大小 9 
第 30 次模拟，误差 0.22423 ，模型大小 10 
第 31 次模拟，误差 0.2127062 ，模型大小 9 
第 32 次模拟，误差 0.4436418 ，模型大小 10 
第 33 次模拟，误差

# n = 200, p = 1000, ISIS-SCAD

In [39]:
# 样本量为n = 200，变量数为p = 1000，SIS-SCAD的模拟
errors_SISSCAD_small = c() # 记录每次模拟的误差
model_sizes_SISSCAD_small = c() # 记录每次模拟产生的模型大小
start_time = Sys.time() # 记录SIS-SCAD small开始时间

for(i in 1:5){ # 测试200个数据集
  
    set.seed(123*i) # 保证生成不同的数据集且具有可重复性
    
    n = 200 # 样本大小
    p = 1000 # 参数维数
    x = matrix(rnorm(n*p, mean=0, sd=1), n, p) # 200*1000 独立同分布标准正态自变量
    x = scale(x) # 标准化
    eps = rnorm(n, mean=0, sd=1.5) # 标准差为1.5的正态噪声

    set.seed(456*i) # 保证生成不同的数据集且具有可重复性
    s = 8 # 真实模型大小
    
    # 生成beta
    u = rbinom(s, 1, 0.4) # 生成 s = 8 个伯努利随机数
    z = rnorm(s, mean=0, sd=1) # 生成 s = 8 个正态分布随机数
    a = 4*log(n)/sqrt(n) # a的定义
    b = ((-1)**(u))*(a + abs(z)) # 长度是 s = 8 的参数向量
    
    # 使用beta生成响应变量y
    y = x[, 1:s] %*% b + eps  # 回归模型
    y = y - mean(y) # 中心化
 
    # 建立SIS-SCAD模型
    modelSIS_small = my_ISIS(x,y, nsis=floor(n/log(n)))
    modelSIS_SCAD_small = tune.fit(x[,modelSIS_small],y,family='gaussian',penalty="SCAD",tune="bic")

    # 计算误差
    b = c(b,rep(0,p-s)) # 真实
    beta_hat = modelSIS_SCAD_small$beta # 估计
    supp_beta_hat = modelSIS_SCAD_small$ix # 支集
    est_error = beta_error(beta_hat,b,supp_beta_hat) # 误差
    errors_SISSCAD_small[i] = est_error # 存储误差
 
    #计算模型大小
    model_sizes_SISSCAD_small[i] = length(supp_beta_hat) # 存储模型大小
    
    # 汇总输出结果
    cat("第",i,"次模拟，误差", errors_SISSCAD_small[i],"，模型大小",model_sizes_SISSCAD_small[i],"\n") # 显示结果
}
end_time = Sys.time() # 记录SIS-SCAD small结束时间

SISSCAD_total_time_small = end_time - start_time # SIS-SCAD small的总耗时
SISSCAD_error_small_median = median(errors_SISSCAD_small) # 误差的中位数
SISSCAD_model_Size_small_median = median(model_sizes_SISSCAD_small) # 模型大小中位数

cat("SIS-SCAD small总耗时",SISSCAD_total_time_small,"秒，误差中位数",SISSCAD_error_small_median,"，模型大小中位数",SISSCAD_model_Size_small_median,"\n")

第 1 次模拟，误差 1.628551 ，模型大小 37 
第 2 次模拟，误差 1.815727 ，模型大小 37 
第 3 次模拟，误差 1.462902 ，模型大小 37 
第 4 次模拟，误差 1.583072 ，模型大小 37 
第 5 次模拟，误差 1.602814 ，模型大小 37 
SIS-SCAD small总耗时 11.81652 秒，误差中位数 1.602814 ，模型大小中位数 37 


# n = 800, p = 20000, SIS-SCAD

In [40]:
errors_SISSCAD_Large = c() 
model_sizes_SISSCAD_Large = c() 
start_time = Sys.time() 
for(i in 1:10){ 

  set.seed(123*i)
  n = 800 
  p = 20000 
  x = matrix(rnorm(n*p, mean=0, sd=1), n, p)
  
  set.seed(456*i)
  s = 18 
  u = rbinom(s, 1, 0.4)
  z = rnorm(s, mean=0, sd=1)
  a = 5*log(n)/sqrt(n)
  b= ((-1)**(u))*(a + abs(z))
  y=x[, 1:s]%*%b + rnorm(n, mean=0, sd=1.5)
  
  modelSIS_Large = SIS(x, y, family='gaussian', iter = FALSE, nsis=(n/log(n)))
  
  modelSCAD = cv.ncvreg(x[,modelSIS_Large$sis.ix0], y, alpha=1, family="gaussian", penalty="SCAD")
  modelNew = ncvreg(x[,modelSIS_Large$sis.ix0], y, alpha=1, family="gaussian", penalty="SCAD", lambda=modelSCAD$lambda.min)
  
  
    # 计算误差
    beta_hat = modelNew$beta[-1,]                
    b = c(b,rep(0,p-length(b)))
    
    est_error = sqrt(sum((beta_hat - b[modelSIS_Large$sis.ix0])^2)+sum((b[-modelSIS_Large$sis.ix0])^2))
    
    
    errors_SISSCAD_Large[i] = est_error
  
  model_sizes_SISSCAD_Large[i] = sum(modelNew$beta[] != 0)-1
   cat("第",i,"次模拟，误差", errors_SISSCAD_Large[i],"，模型大小",model_sizes_SISSCAD_Large[i],"\n") # 显示结果
}
end_time = Sys.time()
SISSCAD_total_time_Large = end_time - start_time
SISSCAD_error_Large_median = median(errors_SISSCAD_Large)
SISSCAD_model_Size_Large_median = median(model_sizes_SISSCAD_Large)


SISSCAD_total_time_Large
SISSCAD_error_Large_median
SISSCAD_model_Size_Large_median

第 1 次模拟，误差 1.632548 ，模型大小 41 
第 2 次模拟，误差 1.384183 ，模型大小 17 
第 3 次模拟，误差 0.2762529 ，模型大小 18 
第 4 次模拟，误差 1.576965 ，模型大小 17 
第 5 次模拟，误差 0.2118433 ，模型大小 18 
第 6 次模拟，误差 1.326115 ，模型大小 39 
第 7 次模拟，误差 0.2112694 ，模型大小 25 
第 8 次模拟，误差 0.3303467 ，模型大小 36 
第 9 次模拟，误差 1.30136 ，模型大小 17 
第 10 次模拟，误差 3.075995 ，模型大小 47 


Time difference of 1.263354 mins

[1] 1.313737

[1] 21.5

# n = 200, p = 1000, LASSO(LARS)

In [16]:
errors_LASSO = c()
model_sizes_LASSO = c()
start_time = Sys.time()
for(i in 1:10){

    set.seed(123*i)
    
    n = 200 
    p = 1000 
    x = matrix(rnorm(n*p, mean=0, sd=1), n, p) 
    x = scale(x) 
    eps = rnorm(n, mean=0, sd=1.5) 

    set.seed(456*i) 
    s = 8 
    
    u = rbinom(s, 1, 0.4) 
    z = rnorm(s, mean=0, sd=1)
    a = 4*log(n)/sqrt(n) 
    b = ((-1)**(u))*(a + abs(z))
    
    y = x[, 1:s] %*% b + eps
    y = y - mean(y)
    
    modelLASSO = lars(x,y,type="lasso",normalize=TRUE,use.Gram=FALSE,max.steps=floor(sqrt(4*p)))

    beta_hat = modelLASSO$beta[length(modelLASSO$df),]
    b=c(b,rep(0,p-s))
    supp_beta_hat = which(beta_hat != 0) # 估计
    beta_hat = beta_hat[supp_beta_hat]
    est_error = beta_error(beta_hat,b,supp_beta_hat) # 误差

    errors_LASSO[i] =  est_error
  
    model_sizes_LASSO[i] = length(supp_beta_hat) 
    print(c(i, errors_LASSO[i], model_sizes_LASSO[i]))

}
end_time = Sys.time()

LASSO_total_time = end_time - start_time
LASSO_error_median = median(errors_LASSO)
LASSO_model_Size_median = median(model_sizes_LASSO)

LASSO_model_Size_median
LASSO_error_median

[1]  1.0000000  0.7263616 59.0000000
[1]  2.0000000  0.7683601 63.0000000
[1]  3.0000000  0.7874259 63.0000000
[1]  4.000000  0.885538 63.000000
[1]  5.0000000  0.8675519 63.0000000
[1]  6.00000  1.04834 63.00000
[1]  7.0000000  0.7358683 63.0000000
[1]  8.0000000  0.8631519 63.0000000
[1]  9.000000  1.146132 59.000000
[1] 10.0000000  0.8197376 63.0000000


[1] 63

[1] 0.8414448

# n = 200, p = 1000, LASSO(LARS) + CV

In [17]:
errors_LASSO = c()
model_sizes_LASSO = c()
start_time = Sys.time()
for(i in 1:5){

    set.seed(123*i)
    
    n = 200 
    p = 1000 
    x = matrix(rnorm(n*p, mean=0, sd=1), n, p) 
    x = scale(x) 
    eps = rnorm(n, mean=0, sd=1.5) 

    set.seed(456*i) 
    s = 8 
    
    u = rbinom(s, 1, 0.4) 
    z = rnorm(s, mean=0, sd=1)
    a = 4*log(n)/sqrt(n) 
    b = ((-1)**(u))*(a + abs(z))
    
    y = x[, 1:s] %*% b + eps
    y = y - mean(y)
    
    cvsol = cv.lars(x,y,type="lasso",plot.it=FALSE,mode="step",use.Gram=FALSE,max.steps=100)
    modelLASSO = lars(x,y,type="lasso",normalize=TRUE,use.Gram=FALSE,max.steps=cvsol$index[which.min(cvsol$cv)])

    beta_hat = modelLASSO$beta[length(modelLASSO$df),]
    b=c(b,rep(0,p-s))
    supp_beta_hat = which(beta_hat != 0) # 估计
    beta_hat = beta_hat[supp_beta_hat]
    est_error = beta_error(beta_hat,b,supp_beta_hat) # 误差

    errors_LASSO[i] =  est_error 
  
    model_sizes_LASSO[i] = length(supp_beta_hat)
    print(c(i, errors_LASSO[i], model_sizes_LASSO[i]))

}
end_time = Sys.time()

LASSO_total_time = end_time - start_time
LASSO_error_median = median(errors_LASSO)
LASSO_model_Size_median = median(model_sizes_LASSO)

LASSO_model_Size_median
LASSO_error_median

[1]  1.0000000  0.7312859 39.0000000
[1]  2.0000000  0.8770849 81.0000000
[1]  3.0000000  0.7936717 32.0000000
[1]  4.000000  0.883164 61.000000
[1]  5.0000000  0.8695911 49.0000000


[1] 49

[1] 0.8695911

# n = 200, p = 1000, SIS-DS

In [18]:
errors_SISDS_small = c() 
model_sizes_SISDS_small = c() 
start_time = Sys.time()
for(i in 1:5){
  
  set.seed(123*i)
  n = 200 
  p = 1000 
  x = matrix(rnorm(n*p, mean=0, sd=1), n, p) 
    
 
  set.seed(456*i)
  s = 8
  u = rbinom(s, 1, 0.4)
  z = rnorm(s, mean=0, sd=1)
  a = 4*log(n)/sqrt(n)
  b= ((-1)**(u))*(a + abs(z))
  y=x[, 1:s]%*%b + rnorm(n, mean=0, sd=1.5)
  

  modelSIS_small = SIS(x, y, family='gaussian', iter = FALSE, nsis=(n/log(n)))
  
  modelDS = dantzig(x[,modelSIS_small$sis.ix0], y)
  
   # 计算误差
    beta_hat = modelDS$BETA0
    beta_hat = beta_hat[,dim(beta_hat)[2]]
    
    model_size = sum(beta_hat != 0)
    
    beta_hat = c(beta_hat,rep(0,p-length(beta_hat)))                
    b = c(b,rep(0,p-length(b)))      
    est_error = sqrt(sum(beta_hat-b)^2)
    
    errors_SISSCAD_small[i] = est_error # 存储误差
  

  a = modelDS$BETA0[, dim(beta_hat)[2]] != 0

    
  errors_SISDS_small[i] = est_error
    
    model_sizes_SISDS_small[i] = model_size
  
  print(c(i, errors_SISDS_small[i], model_sizes_SISDS_small[i])) 
}



end_time = Sys.time()
SISDS_total_time_small = end_time - start_time
SISDS_error_small_median = median(errors_SISDS_small)
SISDS_model_Size_small_median = median(model_sizes_SISDS_small)


SISDS_total_time_small
SISDS_error_small_median
SISDS_model_Size_small_median

compute X^TX and X^y 
start recovering 
lambdamin is  2.326359e-11 
Done! 
[1]  1.00000000  0.08621021 37.00000000
compute X^TX and X^y 
start recovering 
lambdamin is  2.445736 
Done! 
[1]  2.0000000  0.6330811 36.0000000
compute X^TX and X^y 
start recovering 
lambdamin is  6.59696e-13 
Done! 
[1]  3.000000  2.039086 37.000000
compute X^TX and X^y 
start recovering 
lambdamin is  4.478988e-13 
Done! 
[1]  4.000000000  0.006980498 37.000000000
compute X^TX and X^y 
start recovering 
lambdamin is  1.674452 
Done! 
[1]  5.000000  1.165132 35.000000


Time difference of 1.731276 secs

[1] 0.6330811

[1] 37

# n = 800, p = 20000, SIS-DS

In [19]:
errors_SISDS_Large = c() 
model_sizes_SISDS_Large = c() 
start_time = Sys.time()
for(i in 1:5){

  set.seed(123*i) 
  n = 800 
  p = 20000 
  x = matrix(rnorm(n*p, mean=0, sd=1), n, p) 
  

  set.seed(456*i) 
  s = 18
  u = rbinom(s, 1, 0.4)
  z = rnorm(s, mean=0, sd=1)
  a = 5*log(n)/sqrt(n)
  b= ((-1)**(u))*(a + abs(z))
  y=x[, 1:s]%*%b + rnorm(n, mean=0, sd=1.5)
  
 
    modelSIS_Large = SIS(x, y, family='gaussian', iter = FALSE, nsis=(n/log(n)))
    
    modelDS = dantzig(x[,modelSIS_Large$sis.ix0], y,nlambda = 300)
  
   # 计算误差
    beta_hat = modelDS$BETA0
    beta_hat = beta_hat[,dim(beta_hat)[2]]
    
    model_size = sum(beta_hat != 0)
    
    beta_hat = c(beta_hat,rep(0,p-length(beta_hat)))                
    b = c(b,rep(0,p-length(b)))      
    est_error = sqrt(sum(beta_hat-b)^2)
  
    errors_SISDS_Large[i] = est_error 
    
    model_sizes_SISDS_Large[i] = model_size
    
    
    
    print(c(i, errors_SISDS_Large[i], model_sizes_SISDS_Large[i])) 
}
end_time = Sys.time()
SISDS_total_time_Large = end_time - start_time
SISDS_error_Large_median = median(errors_SISDS_Large)
SISDS_model_Size_Large_median = median(model_sizes_SISDS_Large)


SISDS_total_time_Large
SISDS_error_Large_median
SISDS_model_Size_Large_median

compute X^TX and X^y 
start recovering 
lambdamin is  6.036525e-09 
Done! 
[1]   1.000000   2.543266 119.000000
compute X^TX and X^y 
start recovering 
lambdamin is  8.279479e-09 
Done! 
[1]   2.000000   1.077069 119.000000
compute X^TX and X^y 
start recovering 
lambdamin is  2.520486e-10 
Done! 
[1]   3.0000000   0.1325098 119.0000000
compute X^TX and X^y 
start recovering 
lambdamin is  5.145992e-11 
Done! 
[1]   4.0000000   0.6056999 119.0000000
compute X^TX and X^y 
start recovering 
lambdamin is  5.425498e-10 
Done! 
[1]   5.0000000   0.3213144 119.0000000


Time difference of 37.04683 secs

[1] 0.6056999

[1] 119

# n = 200, p = 1000, DS

In [20]:
errors_SISDS_small = c() 
model_sizes_SISDS_small = c() 
start_time = Sys.time()
for(i in 1:1){
  
  set.seed(123*i) 
  n = 200 
  p = 1000 
  x = matrix(rnorm(n*p, mean=0, sd=1), n, p) 
  

  set.seed(456*i) 
  s = 18
  u = rbinom(s, 1, 0.4)
  z = rnorm(s, mean=0, sd=1)
  a = 4*log(n)/sqrt(n)
  b= ((-1)**(u))*(a + abs(z))
  y=x[, 1:s]%*%b + rnorm(n, mean=0, sd=1.5)
  
  modelDS = dantzig(x, y,nlambda=200000)
  
   # 计算误差
    beta_hat = modelDS$BETA0
    beta_hat = beta_hat[,dim(beta_hat)[2]]
    
    model_size = sum(beta_hat != 0)
    
    beta_hat = c(beta_hat,rep(0,p-length(beta_hat)))                
    b = c(b,rep(0,p-length(b)))      
    est_error = sqrt(sum(beta_hat-b)^2)
    
    errors_SISSCAD_small[i] = est_error # 存储误差
  

  a = modelDS$BETA0[, dim(beta_hat)[2]] != 0

  errors_SISDS_small[i] = est_error 
    
    model_sizes_SISDS_small[i] = model_size
  
  print(c(i, errors_SISDS_small[i], model_sizes_SISDS_small[i])) 

}

end_time = Sys.time()
SISDS_total_time_small = end_time - start_time
SISDS_error_small_median = median(errors_SISDS_small)
SISDS_model_Size_small_median = median(model_sizes_SISDS_small)


SISDS_total_time_small
SISDS_error_small_median
SISDS_model_Size_small_median

compute X^TX and X^y 
start recovering 
lambdamin is  0.005627696 
Done! 
[1]   1.0000000   0.9236602 199.0000000


Time difference of 1.244969 mins

[1] 0.9236602

[1] 199

# Leukaemia

In [33]:
data("leukemia.train", package = "SIS")
data("leukemia.test", package = "SIS")
y1 = leukemia.train[, dim(leukemia.train)[2]]
x1 = as.matrix(leukemia.train[, -dim(leukemia.train)[2]])
y2 = leukemia.test[, dim(leukemia.test)[2]]
x2 = as.matrix(leukemia.test[, -dim(leukemia.test)[2]])

x = rbind(x1, x2)
y = c(y1, y2)

penalty = "SCAD"
tune = "bic"
nsis = 100
q = 0.95
st = FALSE
tot.sim = 10
results.leukemia.1 = matrix(0, nrow=tot.sim, ncol=3)

for(randSeed in 1:tot.sim){
set.seed(randSeed)

n = dim(x)[1]; aux = 1:n
ind.train1 = sample(aux[y == 0], 23, replace = FALSE)
ind.train2 = sample(aux[y == 1], 13, replace = FALSE)
ind.train = c(ind.train1, ind.train2)
x.train = scale(x[ind.train,])
y.train = y[ind.train]
ind.test1 = setdiff(aux[y == 0], ind.train1)
ind.test2 = setdiff(aux[y == 1], ind.train2)
ind.test = c(ind.test1, ind.test2)
x.test = scale(x[ind.test,])
y.test = y[ind.test]

r = SIS(x.train, y.train , family="binomial", penalty=penalty, tune=tune, nsis=nsis, iter=FALSE, standardize=st)
train.error = length(which(y.train!= predict(r, x.train, type="class")))
test.error = length(which(y.test!= predict(r, x.test, type="class")))
results.leukemia.1[randSeed,] = c(train.error,test.error,length(r$ix))
}

results.leukemia = matrix(0, nrow=1, ncol=6)
results.leukemia[1,c(1,3,5)] = apply(results.leukemia.1,2,median); results.leukemia[1,c(2,4,6)] = apply(results.leukemia.1,2,IQR)/1.34
results.leukemia

11,4.104478,11,4.477612,1,1.30597
